# Perfomance PACE (paper - technical evaluation)

PACE was designed/implemented to be used in a notebook environment. Hence, a notebook should be used to evaluate its performance.

We need to evaluate three spects:
1. Missingness computation (time in sec)
2. Visualisation (time in sec)
3. RAM (`psutil.virtual(memory)`)

The results of the evaluation are written to a csv file. The filename contains the visualisation package that was used and a timestamp `outputfile_<package>_<yyyymmdd-HHMMSS>`. The config file that was used  for the evaluation is saved with the timestamp `config_<yyyymmdd-HHMMSS>.yaml`. The config file contains the parameters required to run the evaluation and a comment with the `git commit id`.

<line>


### To do:

- [x] add timings we're interested in to results
- [ ] add RAM to evaluation -> where's the right point to 
- [ ] different data types for all the patterns
- [x] write results to file
- [ ] Write script to write dataset from csv to postgres

### Notes:

- can't find an obvious or straight forward way to save bokeh figure the way we use it
- possibly good enough to provile without saving figure -> check the time for the pace plot function

In [1]:
from pace.membership import Membership
from pace.plots import PlotSession
from utils import generate_pattern, eval_data, set_seed
from datetime import datetime, time
import psutil
import yaml
import pandas as pd
import csv
from datetime import datetime
from pathlib import Path
import subprocess
# Yaml loaders and dumpers
from ruamel.yaml.main import round_trip_load as yaml_load, round_trip_dump as yaml_dump
import io
import cProfile
import pstats
import pandas as pd
import re

In [2]:
# just for debugging
import upsetplot

## Save copy of config file

Add a comment line with git commit hash and then copy content of `config.yaml`. 

In [4]:
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
commit = subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode().strip()

In [5]:
config_in = Path.cwd() / "config.yaml"
config_out = Path.cwd() / "results" /f"config_{timestamp}.yml"

with open(config_in, 'r') as f:
    config_content = yaml_load(f)

with open(config_out, "w") as f:
    f.write(f"# git commit hash: {commit} \n")
    f.write(yaml_dump(config_content))


## Load simulation parameters

In [6]:
config_yaml = open("config.yaml")
config = yaml.load(config_yaml, Loader=yaml.FullLoader)

In [7]:
num_rows = config["dataset"]["num_rows"]
num_cols = config["dataset"]["num_cols"]
num_int = config["dataset"]["intersections"]  # number of intersections
data_type = config["dataset"]["datatype"]
seed = config["seed"][0]
patterns = config["patterns"]
filename = config["output"]["filename"]
package = "pace"

## Files

In [8]:
path = Path.cwd() / "evaluation"
output_file = Path.cwd() / "results" / f"{filename}_{package}_{timestamp}.csv"
profile_file = Path.cwd() / "profile.csv"

## Functions

In [9]:
def profile_to_csv(profile, file="profile.csv"):
    """Writes the result of the profiling a csv file.
    
    Parameters
    ----------
    profile : cProfile.Profile
        text stream 
    file : str
        path to csv file to which we write the profiling results from text stream
    """
    result = io.StringIO()
    pstats.Stats(profile, stream=result).print_stats()
    result = result.getvalue()
    
    # chop string into a csv-like buffer
    result='ncalls'+result.split('ncalls')[-1]
    result='\n'.join([','.join(line.rstrip().split(None,5)) for line in result.split('\n')])

    # write to csv file
    with open(file, 'w') as f:
        f.write(result)
        f.close()


    

In [10]:
def extract_function_name(string):
    """Extracts the function name from a string. 
    
    Assumes that the function name is between parenthesis. 
    Applied on the column 'filename:lineno(function)'
    
    Parameters
    ----------
    string : str
        string with the value of the above described column
    
    Returns
    -------
    function_name : str
        name of the function without path or line number
    """
    result = re.search(r"\((.*?)\)",string)
    if result:
        return result.group(1)
    else:
        return "-900"

## Generate data and evaluate

In [11]:
if package == "upset":
    data_func = ""

with open(output_file, "w") as csvfile:
    try:
        w = csv.writer(csvfile, delimiter=",")
        w.writerow(
            [
                "Package",
                "Pattern",
                "Num Rows",
                "Num Cols",
                "Num Intersections",
                "Tims (s) Missingness",
                "Tims (s) Total",
                "RAM",
            ]
        )
        # set seed
        set_seed(seed)
        # run evaluation
        for pattern in patterns:
            for dtype in data_type:
                for inter in num_int:
                    for row in num_rows:
                        for col in num_cols:
                            # step 1: generate data
                            df = generate_pattern(
                                pattern, row, col, inter, dtype
                            )
                            # step 2: evaluate data
                            pr = cProfile.Profile()
                            pr.enable()
                            vir_mem = eval_data(df, package)
                            pr.disable()
                            profile_to_csv(pr, file=profile_file)
                            # step 3: load profile and extract timings we're interested in
                            df = pd.read_csv(profile_file)
                            df["function"] = df["filename:lineno(function)"].apply(lambda row: extract_function_name(row))
                            # step 4: add timings to result
                            time_total = (df["cumtime"][df["function"] == "eval_data"]).values[0]
                            time_missingness = (df["cumtime"][df["function"] == "compute_missingness"]).values[0]
                            results = [package, pattern, row, col, inter,time_missingness, time_total, 1]
                            w.writerow(results)
    except:
        raise

In [12]:
df.sort_values(by=["cumtime"], ascending=False)

,ncalls,tottime,percall,cumtime,percall.1,filename:lineno(function),function
1289,1,0.0,0.0,0.101,0.101,/Users/pwochner/Projects/Visulisation/visualis...,eval_data
1277,2,0.0,0.0,0.089,0.045,/Users/pwochner/Projects/Visulisation/visualis...,run_notebook_hook
1104,1,0.0,0.0,0.083,0.083,/Users/pwochner/Projects/Visulisation/visualis...,add_plot
1285,1,0.0,0.0,0.082,0.082,/Users/pwochner/Projects/Visulisation/visualis...,show
1280,1,0.0,0.0,0.082,0.082,/Users/pwochner/Projects/Visulisation/visualis...,show_app
...,...,...,...,...,...,...,...
609,2,0.0,0.0,0.000,0.000,/Users/pwochner/Projects/Visulisation/visualis...,bincount
608,1,0.0,0.0,0.000,0.000,/Users/pwochner/Projects/Visulisation/visualis...,can_cast
607,10,0.0,0.0,0.000,0.000,/Users/pwochner/Projects/Visulisation/visualis...,concatenate
605,2,0.0,0.0,0.000,0.000,/Users/pwochner/Projects/Visulisation/visualis...,_check


In [14]:
results

['pace', 'monotone', 10, 5, 10, 0.006, 0.101, 1]